In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
phases = {}
G = {}
for i in range(1,12): 
  var_name = "phase" + str(i)
  file_name = "https://raw.githubusercontent.com/ragini30/Networks-Homework/main/" + var_name + ".csv"
  phases[i] = pd.read_csv(file_name, index_col = ["players"])
  phases[i].columns = "n" + phases[i].columns
  phases[i].index = phases[i].columns
  G[i] = nx.from_pandas_adjacency(phases[i])
  G[i].name = var_name

In [3]:
phase_list = list(phases.keys())
phase_list

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [4]:
from itertools import product
playernames = [f'n{i}' for i in range(1,111)]
df_PlayerPhase = pd.DataFrame(list(product(phase_list, playernames)), columns=['phase', 'player'])
df_PlayerPhase['player'] = df_PlayerPhase['player'].astype("string")
df_PlayerPhase['degree_cent'] = np.nan
df_PlayerPhase['betweenness'] = np.nan
df_PlayerPhase['eigenvector'] = np.nan

def playerphase_stat(player, phase, stat):
    return df_PlayerPhase.loc[(df_PlayerPhase['phase'] == phase) 
                                & (df_PlayerPhase['player'] == player)][stat].iloc[0]

for p in phase_list:
    dc = nx.degree_centrality(G[p])
    df_PlayerPhase.loc[df_PlayerPhase['phase'] == p, 'degree_cent'] = \
    df_PlayerPhase.loc[df_PlayerPhase['phase'] == p, 'player'].map(dc)

for p in phase_list:
    bc = nx.betweenness_centrality(G[p], normalized = True)
    df_PlayerPhase.loc[df_PlayerPhase['phase'] == p, 'betweenness'] = \
    df_PlayerPhase.loc[df_PlayerPhase['phase'] == p, 'player'].map(bc)

for p in phase_list:
    bc = nx.eigenvector_centrality(G[p])
    df_PlayerPhase.loc[df_PlayerPhase['phase'] == p, 'eigenvector'] = \
    df_PlayerPhase.loc[df_PlayerPhase['phase'] == p, 'player'].map(bc)

playerphase_stat('n3', 1, 'degree_cent')

0.14285714285714285

In [5]:
# fill missing numeric values
df_PlayerPhase = df_PlayerPhase.fillna(value=0)
df_PlayerPhase

,phase,player,degree_cent,betweenness,eigenvector
0,1,n1,0.857143,0.906593,0.626451
1,1,n2,0.071429,0.000000,0.160482
2,1,n3,0.142857,0.000000,0.253842
3,1,n4,0.071429,0.000000,0.160482
4,1,n5,0.071429,0.000000,0.160482
...,...,...,...,...,...
1205,11,n106,0.000000,0.000000,0.000000
1206,11,n107,0.000000,0.000000,0.000000
1207,11,n108,0.000000,0.000000,0.000000
1208,11,n109,0.000000,0.000000,0.000000


In [6]:
df_PlayerNames = pd.read_csv('./CAVIAR/CAVIAR_names.csv')
df_PlayerNames

,player,full_name,description
0,n1,Daniel Serero,Mastermind of the network.
1,n3,Pierre Perlini,"Principal lieutenant of Serero, he executes Se..."
2,n5,Richard Gleeson,Charged with recuperating the marijuana.
3,n6,Gaspard Lino,Broker in Spain.
4,n8,Bruno de Quinzio,Charged with recuperating the marijuana.
5,n11,Samir Rabbat,Provider in Morocco.
6,n12,Ernesto Morales,"Principal organizer of the cocaine import, int..."
7,n16,Johnny Pacheco,Cocaine provider.
8,n17,Oscar Nieri,The handyman of Morales.
9,n33,Ricardo Negrinotti,Was taking possession of the cocaine in the US...


In [7]:
df_PlayerPhase = df_PlayerPhase.merge(df_PlayerNames, on='player', how='left')
# fill resulting missing text values
df_PlayerPhase = df_PlayerPhase.fillna(value='')
df_PlayerPhase

,phase,player,degree_cent,betweenness,eigenvector,full_name,description
0,1,n1,0.857143,0.906593,0.626451,Daniel Serero,Mastermind of the network.
1,1,n2,0.071429,0.000000,0.160482,,
2,1,n3,0.142857,0.000000,0.253842,Pierre Perlini,"Principal lieutenant of Serero, he executes Se..."
3,1,n4,0.071429,0.000000,0.160482,,
4,1,n5,0.071429,0.000000,0.160482,Richard Gleeson,Charged with recuperating the marijuana.
...,...,...,...,...,...,...,...
1205,11,n106,0.000000,0.000000,0.000000,Beverly Ashton,"Spouse of Lino, transports money and documents."
1206,11,n107,0.000000,0.000000,0.000000,,
1207,11,n108,0.000000,0.000000,0.000000,,
1208,11,n109,0.000000,0.000000,0.000000,,


In [8]:
# replace any pre-existing copy 
fh = open(r'../data/CAVIARn.js','w')
fh.write('')
fh.close()

In [9]:
for i in phase_list:
    df = df_PlayerPhase[df_PlayerPhase['phase'] == i].copy()
    df = df[df['player'].isin(list(G[i].nodes))]
    df['key_player'] = 1
    df['key_player'] = df['key_player'] * np.minimum(df['full_name'].str.len(), 1)
    df.rename(columns={'player': 'id'}, inplace=True)

    nodestr = df.to_json(orient='records', indent=2)

    dfl = pd.DataFrame(G[i].edges.data(), columns=['source', 'target', 'weight'])
    dfl['weight'] = [d.get('weight') for d in dfl['weight']]

    linkstr = dfl.to_json(orient='records', indent=2)

    dset_str = (f'phase{i} = ({{\n'
                f'  nodes: \n'
                f'{nodestr},'
                f'\n  links: \n'
                f'{linkstr},\n}})\n\n')

    fh = open(r'../data/CAVIARn.js','a')
    fh.write(dset_str)
    fh.close()